# Limpieza de datos

## 1. Generación de datos faltantes

## 2. agrupación de datos y eliminación de variables

En este caso vamos a proceder a primero hacer una correlación con las compras y los artículos de cada cliente, para generar un dataframe, que nos resuma los artículos comprados Por cliente

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import preprocessing as ps
from difflib import SequenceMatcher
import csv

In [2]:
client=pd.read_csv("Data/customers.csv")
sells=pd.read_csv("Data/transactions_train.csv")

In [26]:
count_costumer=sells["customer_id"].unique().tolist()
filename=f"Data/newsales.csv"
with open(filename,"a",newline="") as csvfile:
    writer=csv.DictWriter(csvfile,fieldnames=["client_id","articles_id"])
    for i in count_costumer:
        d={}
        d["client_id"]=i
        d["articles_id"]=str(sells.loc[sells["customer_id"]==i,["article_id"]]["article_id"].tolist())
        writer.writerow(d)
    cvsfile.close()

KeyboardInterrupt: 